# SBTi-Finance Tool - Target Reporting
This notebook is an example of how to use the tool to calculate your portfolio's temperature scores and to save results to be submitted to SBTi's Target Validation Team (TVT) for validation and approval of your own Science Based Target.

Please see the [methodology](https://sciencebasedtargets.org/wp-content/uploads/2020/07/Temperature-Scoring-Beta-Methodology.pdf), [guidance](https://sciencebasedtargets.org/financial-institutions) and the [technical documentation](http://getting-started.sbti-tool.org/) for more details on how to calculate portfolio temperature scores and how to submit targets to the SBTi for validation and approval. 

See 1_analysis_example (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/1_analysis_example.ipynb) or [Github](https://github.com/OFBDABV/SBTi/blob/master/examples/1_analysis_example.ipynb)) for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 

## Setting up
This section imports SBTi tools, import data providers, and load the portfolio.

In [ ]:
!pip install SBTi

In [1]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore
from SBTi.interfaces import ETimeFrames, EScope
import pandas as pd

### Download resources for Google Colab environment


In [2]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/example_portfolio.csv", "data/example_portfolio.csv")
if not os.path.isfile("utils.py"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/utils.py", "utils.py")

try:  # Import statement when run in Google Colabs
    from utils import  print_aggregations, print_grouped_scores, anonymize, print_percentage_default_scores
except:  # Import statement when run locally
    from examples.utils import print_aggregations, print_grouped_scores, anonymize, print_percentage_default_scores

Specify the data files to use (portfolio and company data). In this example we are using dummy data. 

To change the files, first upload (when using Google Colab) or place the files to the data folder (when running locally).
Next, change the file locations between the quotes below.

In [3]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")             # adjust the path according to your data
portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1") # adjust the path according to your data
companies = SBTi.utils.dataframe_to_portfolio(portfolio)
portfolio

,company_name,company_id,company_isin,weights,investment_value,engagement_target
0,Company AG,US0079031078,US0079031078,6.36%,35000000,True
1,Company AH,US00724F1012,US00724F1012,1.82%,10000000,False
2,Company AI,FR0000125338,FR0000125338,1.82%,10000000,False
3,Company AJ,US17275R1023,US17275R1023,1.82%,10000000,False
4,Company AK,CH0198251305,CH0198251305,1.82%,10000000,False
5,Company AL,US1266501006,US1266501006,1.82%,10000000,False
6,Company AM,FR0000120644,FR0000120644,1.82%,10000000,False
7,Company AN,US24703L1035,US24703L1035,1.82%,10000000,False
8,Company AO,TW0002308004,TW0002308004,1.82%,10000000,False
9,Company AP,FR0000120321,FR0000120321,1.82%,10000000,False


## Select calculation options
Configure computational settings for the temperature score. 
You can change the default score (fallback score in the code) and aggregation method below. The values specified here are the recommended defaults by the SBTi. If you want to overwrite the recommended settings see the comments for all the options per setting

In [4]:
# Settings                                              # Other Options: 
time_frames = [SBTi.interfaces.ETimeFrames.MID]         # reporting to TVT requires 'MID'
scopes = [EScope.S1S2, EScope.S1S2S3]                   # reporting to TVT requires 'S1S2' and 'S1S2S3'
fallback_score = 3.2                                    # The options of the default score are 3.2, 3.9 or 4.5
aggregation_method = PortfolioAggregationMethod.WATS    # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
grouping = ['sector']                                   # reporting to TVT requires 'sector'

In [5]:
temperature_score = TemperatureScore(
    time_frames=time_frames,
    scopes=scopes,
    fallback_score=fallback_score,
    aggregation_method=aggregation_method,
    grouping=grouping
)

## Calculate company and portfolio temperature scores

In [6]:
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
amended_portfolio.head(5) # display top 5 rows

,company_id,time_frame,scope,target_type,intensity_metric,coverage_s1,coverage_s2,coverage_s3,reduction_ambition,base_year,...,annual_reduction_rate,slope,samplesize,model,variable,param,intercept,r2,temperature_score,temperature_results
0,US0079031078,MID,S1S2,Absolute,nan,1.0,1.0,NaN,0.30,2018.0,...,0.042857,slope15,128.0,4.0,Emissions|Kyoto Gases,-0.312329,2.697261,0.829523,1.36,0.000000
2,US0079031078,MID,S1S2S3,absolute,None,NaN,NaN,NaN,NaN,NaN,...,NaN,slope15,NaN,NaN,NaN,NaN,NaN,NaN,2.70,0.727326
3,US00724F1012,MID,S1S2,intensity,Revenue,1.0,1.0,0.9,0.24,2014.0,...,0.015000,slope15,128.0,4.0,INT.emKyoto_gdp,-0.533878,4.107377,0.826692,3.31,0.000000
5,US00724F1012,MID,S1S2S3,absolute,None,NaN,NaN,NaN,NaN,NaN,...,NaN,slope15,NaN,NaN,NaN,NaN,NaN,NaN,3.31,0.000000
6,FR0000125338,MID,S1S2,absolute,None,NaN,NaN,NaN,NaN,NaN,...,NaN,slope15,128.0,4.0,Emissions|Kyoto Gases,-0.312329,2.697261,0.829523,3.20,1.000000


## Checks
In this example, the following scores are calculated:
- Portfolio temperature score
- Score for each sector
- Percentage covered by targets vs. default scores
- Portfolio coverage

In [7]:
# Portfolio temperature score
print_aggregations(aggregated_scores)

Timeframe  Scope      Temp score
mid        S1S2       2.63
mid        S1S2S3     2.88


In [8]:
# Score per sector
print_grouped_scores(aggregated_scores)


                         Temp score
mid - S1S2
Communication Services   3.20
Consumer Discretionary   2.44
Consumer Staples         3.20
Energy                   3.20
Financials               3.20
Health Care              2.70
Industrials              2.36
Information Technology   1.32
Materials                3.20
Real Estate              3.28
Utilities                2.58

                         Temp score
mid - S1S2S3
Communication Services   3.20
Consumer Discretionary   3.08
Consumer Staples         3.20
Energy                   3.20
Financials               3.11
Health Care              3.05
Industrials              2.72
Information Technology   2.15
Materials                3.20
Real Estate              3.21
Utilities                2.73


In [9]:
# Percentage of score based on default score
print_percentage_default_scores(aggregated_scores)

Timeframe  Scope      % Default score
mid        S1S2       66.36
mid        S1S2S3     78.02


In [10]:
# Portfolio coverage: Percentage of SBTs in portfolio
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
print("Portfolio coverage is {c:.2f}%".format(c=coverage))

Portfolio coverage is 35.45%


## Save anonymized data for SBTi target validation
In order for the targets to be validated by SBTi, you can save your data locally. By running the anonymize function, you can replace company identifiers with meaningless substitutes.

In [11]:
portfolio, provider = anonymize(portfolio, provider)

In order to store the portfolio and provider data locally, two options apply:
1. You are running the SBTi tool locally or from Google Colab
2. You are running the SBTi tool from a Docker container

If you run the SBTi tool locally or from Google Colab, you:
- Specify and filenames in the cell below
- Run the cell below

In [12]:
portfolio_filename = 'portfolio.xlsx'
provider_filename = 'provider.xlsx'
portfolio.to_excel(portfolio_filename, index=False)

writer = pd.ExcelWriter(provider_filename, engine='openpyxl')
provider.data['fundamental_data'].to_excel(writer, sheet_name='fundamental_data')
provider.data['target_data'].to_excel(writer, sheet_name='target_data')
writer.save()

If you run the SBTi tool locally, you find the output files in the root folder of this notebook

If you run the SBTi tool from Google Colab, you:
- Click on the files icon in the left pane
- Click the three dots that appear after hovering over the file
- Download the file to your local machine